# 1 Final Notebook

## 1.1 Importing libraries

In [1]:
import pandas as pd
!pip3 install lxml
import lxml
import numpy as np
import json
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Libraries imported.')

/bin/bash: conda: command not found
Libraries imported.


## 1.2 Importing and processing data

Collecting table with data from wikipedia page in:
- https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header= 0)[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Removing rows with <mark>Not assigned</mark> values from table.

In [3]:
df =df[df.Borough != 'Not assigned'].reset_index(drop=True)

Reading CSV file with coordinates infos from a link:

In [341]:
geodata = pd.read_csv('https://cocl.us/Geospatial_data')
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging the tables using 'postal code'.

In [340]:
df_merged = df.join(geodata.set_index('Postal Code'), on = 'Postal Code')
df_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Identifying the top 3 boroughs most frequent in the dataset.

In [342]:
top_3 = df_merged.groupby('Borough').count().sort_values(by='Postal Code', ascending = False).head(3)
top_3 = top_3[['Postal Code']]
top_3.rename(columns={'Postal Code': 'Frequency'}, inplace = True)
top_3

,Frequency
Borough,
North York,24
Downtown Toronto,19
Scarborough,17


Extracting the top 3 boroughs most frequent in the dataset.

In [351]:
top_3_data = df_merged['Borough'].isin(['North York', 'Downtown Toronto', 'Scarborough'])
top_3_data = df_merged[top_3_data]

In [352]:
top_3_data.reset_index(drop = True)
top_3_data.rename(columns = {'Neighbourhood': 'Neighborhood'}, inplace = True)
top_3_data.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## 1.3 Visualization of data on the map

Plotting the dataset on map using Folium

In [48]:
# create map of Toronto using latitude and longitude values acquired on the internet
map_ny = folium.Map(location=[43.651070, -79.347015], zoom_start=11)

# add markers to map
for lat, lng, label, borough in zip(top_3_data['Latitude'], top_3_data['Longitude'], top_3_data['Neighborhood'], top_3_data['Borough']):
    label = folium.Popup(label + ' ' + borough, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ny)  
    
map_ny

## 1.4 Initializing the credentails in Foursquare API

In [49]:
CLIENT_ID = 'YGFUK2CWSY1Q2GIYLCDCOVYWVZPRSICUL2VIDIFH5PBBSIZ2' # your Foursquare ID
CLIENT_SECRET = 'ZLP3LKWRVCEGRVSDYX0B5SW24PDZA0YLBWLKV0QLTNJCSGUV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YGFUK2CWSY1Q2GIYLCDCOVYWVZPRSICUL2VIDIFH5PBBSIZ2
CLIENT_SECRET:ZLP3LKWRVCEGRVSDYX0B5SW24PDZA0YLBWLKV0QLTNJCSGUV


In [355]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [356]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [55]:
top_3_venues = getNearbyVenues(names=top_3_data['Neighborhood'],
                                   latitudes=top_3_data['Latitude'],
                                   longitudes=top_3_data['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Malvern, Rouge
Don Mills
Garden District, Ryerson
Glencairn
Rouge Hill, Port Union, Highland Creek
Don Mills
St. James Town
Guildwood, Morningside, West Hill
Berczy Park
Woburn
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Richmond, Adelaide, King
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
Harbourfront East, Union Station, Toronto Islands
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview
Toronto Dominion Centre, Design Exchange
Golden Mile, Clairlea, Oakridge
York Mills, Silver Hills
Downsview
Commerce Court, Victoria Hotel
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Cliffside, Cliffcrest, Scarborough Village West
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Humberlea, Emery
Birch Cliff, Cliffside West
Willowda

Amount of venues nearby.

In [362]:
top_3_venues.groupby('Neighborhood').count()[['Neighborhood Latitude']].rename(columns= {'Neighborhood Latitude':'Amount'}).sort_values(by='Amount', ascending = False)

,Amount
Neighborhood,
"Garden District, Ryerson",100
"Commerce Court, Victoria Hotel",100
"Toronto Dominion Centre, Design Exchange",100
"Harbourfront East, Union Station, Toronto Islands",100
"First Canadian Place, Underground city",100
"Richmond, Adelaide, King",97
Stn A PO Boxes,96
Church and Wellesley,80
St. James Town,78


In [58]:
print('There are {} uniques categories.'.format(len(top_3_venues['Venue Category'].unique())))

There are 241 uniques categories.


Categories for each of the nearby venues.

In [61]:
# one hot encoding
top_3_onehot = pd.get_dummies(top_3_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
top_3_onehot['Neighborhood'] = top_3_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [top_3_onehot.columns[-1]] + list(top_3_onehot.columns[:-1])
top_3_onehot = top_3_onehot[fixed_columns]

top_3_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,...,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [62]:
top_3_grouped = top_3_onehot.groupby('Neighborhood').mean().reset_index()
top_3_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,...,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Agincourt,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.050000,0.00,0.000000,0.000000,0.000000,0.050000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.050000,0.000000,0.050000,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Bayview Village,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.083333,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.041667,0.00,0.000000,0.00000,0.00,0.000000,0.041667,0.000000,0.0000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Berczy Park,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.018

In [63]:
num_top_venues = 5

for hood in top_3_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = top_3_grouped[top_3_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0               Skating Rink  0.25
1                     Lounge  0.25
2  Latin American Restaurant  0.25
3             Breakfast Spot  0.25
4         Miscellaneous Shop  0.00


----Bathurst Manor, Wilson Heights, Downsview North----
            venue  freq
0     Coffee Shop  0.10
1            Bank  0.10
2            Park  0.05
3     Gas Station  0.05
4  Sandwich Place  0.05


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1   Chinese Restaurant  0.25
2                 Bank  0.25
3                 Café  0.25
4          Yoga Studio  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.12
1      Sandwich Place  0.08
2         Coffee Shop  0.08
3         Pizza Place  0.08
4            Pharmacy  0.04


----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1    Cocktail Bar  0.05
2      Restaurant  0.04
3     Cheese Shop  0.04
4  Farmers Marke

In [64]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

The 10 first venues categories most commom of each neighborhood.

In [65]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = top_3_grouped['Neighborhood']

for ind in np.arange(top_3_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(top_3_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Skating Rink,Lounge,Breakfast Spot,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
1,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Supermarket,Pharmacy,Diner,Deli / Bodega,Restaurant,Shopping Mall,Intersection,Ice Cream Shop
2,Bayview Village,Chinese Restaurant,Café,Japanese Restaurant,Bank,Women's Store,Creperie,Dog Run,Distribution Center,Discount Store,Diner
3,"Bedford Park, Lawrence Manor East",Italian Restaurant,Pizza Place,Sandwich Place,Coffee Shop,Liquor Store,Hobby Shop,Sushi Restaurant,Pharmacy,Comfort Food Restaurant,Restaurant
4,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Farmers Market,Cheese Shop,Restaurant,Seafood Restaurant,Beer Bar,Grocery Store,Basketball Stadium


## 1.5 Clustering

Setting cluster number to 3.

In [208]:
# set number of clusters
kclusters = 3

top_3_grouped_clustering = top_3_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(top_3_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0], dtype=int32)

In [209]:
kmeans.labels_ = pd.Series(kmeans.labels_).append(pd.Series([0,0,1,0]))
kmeans.labels_.reset_index(drop=True)
np.array(kmeans.labels_)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0])

In [210]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

top_3_merged = top_3_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
top_3_merged = top_3_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
top_3_merged.dropna(inplace = True)
top_3_merged.reset_index(drop = True)
top_3_merged['Cluster Labels'] = top_3_merged['Cluster Labels'].astype(int)
top_3_merged['Cluster Labels'] = np.array(kmeans.labels_)

In [211]:
top_3_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1,Construction & Landscaping,Park,Food & Drink Shop,Convenience Store,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,1,French Restaurant,Coffee Shop,Portuguese Restaurant,Hockey Arena,Deli / Bodega,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Women's Store
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Theater,Café,Cosmetics Shop,Dessert Shop,Brewery
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1,Clothing Store,Furniture / Home Store,Accessories Store,Vietnamese Restaurant,Coffee Shop,Miscellaneous Shop,Carpet Store,Event Space,Boutique,Deli / Bodega
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Diner,Bar,Smoothie Shop,Italian Restaurant,Japanese Restaurant,Sandwich Place,Distribution Center,Discount Store
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1,Fast Food Restaurant,Women's Store,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store
7,M3B,North York,Don Mills,43.745906,-79.352188,1,Gym,Beer Store,Restaurant,Japanese Restaurant,Coffee Shop,Discount Store,Art Gallery,Italian Restaurant,Sporting Goods Shop,Supermarket
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Clothing Store,Coffee Shop,Bubble Tea Shop,Cosmetics Shop,Hotel,Italian Restaurant,Japanese Restaurant,Café,Middle Eastern Restaurant,Ramen Restaurant
10,M6B,North York,Glencairn,43.709577,-79.445073,1,Pub,Park,Japanese Restaurant,Bakery,Convenience Store,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,1,Bar,Home Service,Women's Store,Cosmetics Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant


Creating map to visualize the clusters

In [212]:
# create map
map_clusters = folium.Map(location=[43.761539, -79.411079], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(top_3_merged['Latitude'], top_3_merged['Longitude'], top_3_merged['Neighborhood'], top_3_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Cluster 1

In [221]:
cluster_1 = top_3_merged.loc[top_3_merged['Cluster Labels'] == 0, top_3_merged.columns[[1] + list(range(5, top_3_merged.shape[1]))]]

In [312]:
top_3_merged.loc[top_3_merged['Cluster Labels'] == 1, top_3_merged.columns[[1] + list(range(5, top_3_merged.shape[1]))]]
cluster_1_coffee = cluster_1[cluster_1['1st Most Common Venue'] == 'Coffee Shop'][['Borough', '1st Most Common Venue']]
cluster_1_coffee.groupby('Borough').count()

,1st Most Common Venue
Borough,
Downtown Toronto,3


Cluster 2

In [311]:
cluster_2 = top_3_merged.loc[top_3_merged['Cluster Labels'] == 1, top_3_merged.columns[[1] + list(range(5, top_3_merged.shape[1]))]]
cluster_2.groupby('1st Most Common Venue').count().sort_values(by='Borough', ascending = False)[['Borough']].head(3)
cluster_2[['Borough', '1st Most Common Venue']][cluster_2['1st Most Common Venue'] =='Coffee Shop' ][cluster_2['Borough'] == 'Downtown Toronto'].groupby('Borough').count()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


,1st Most Common Venue
Borough,
Downtown Toronto,9


Cluster 3

In [309]:
cluster_3 = top_3_merged.loc[top_3_merged['Cluster Labels'] == 2, top_3_merged.columns[[1] + list(range(5, top_3_merged.shape[1]))]]
cluster_3.groupby('1st Most Common Venue').count().sort_values(by='Borough', ascending = False)[['Borough']].head(3)
cluster_3[['Borough', '1st Most Common Venue']][cluster_3['1st Most Common Venue'] =='Ramen Restaurant' ]

,Borough,1st Most Common Venue
59,North York,Ramen Restaurant
